In [3]:
import glob
import os
import re

import librosa
import torch
import numpy as np

SEC_PER_MIN = 60

# Generating Lyric Label
    
    :format: name.label

In [4]:
def generate_lyrics_label():
    for filename in glob.glob(os.path.join(os.getcwd() + '\Model_Data\lyrics', '*.txt')):
        with open(filename, 'r') as r:
            lines = r.readlines()
            label_list = [0] * 2 * SEC_PER_MIN 
            for line in lines:
                timestamp = re.split(r'\[|\]', line)[1]
                time = re.split(r'[.:]', timestamp)
                # time format: [min, sec, minisec]
                if int(time[0]) < 2:
                    sec = int(time[1]) + int(time[0]) * SEC_PER_MIN 
                    label_list[sec] = 1
            with open(filename.split('.')[0] + ".label", 'w') as w:
                w.write(' '.join(str(s) for s in label_list))
                w.close()
        r.close()

generate_lyrics_label()

# Zip .wav file with labels
    
    :output: 
        label_torch (tensor of lyric label)
        reg_wav_list (wav, sr) from librosa load
        iso_wav_list (wav, sr) from librosa load         

In [11]:
def zip_label_wav():
    reg_dir = os.getcwd() + '/Model_Data/songs/vocal_reg/'
    iso_dir = os.getcwd() + '/Model_Data/songs/vocal_iso/'
    label_dir = os.getcwd() + '/Model_Data/lyrics/'
    reg_wav_list = []
    iso_wav_list = []
    label_list = []
    # .label file
    for filename in os.scandir(reg_dir):
        with open(label_dir + filename.name.split('.')[0] + '.label', 'r') as r:
            line = r.readline()
            label_list.append(str(line).split(' '))
        r.close()
    label_torch = torch.from_numpy(np.array(label_list, dtype=np.float32))
    print(label_torch.size())
    # .wav file
    for filename in os.scandir(reg_dir):
        print("Loading file:", filename.name)
        reg_wav, reg_sr = librosa.load(filename.path, duration=120)
        iso_wav, iso_sr = librosa.load(iso_dir + filename.name, duration=120)
        reg_wav_list.append((reg_wav, reg_sr))
        iso_wav_list.append((iso_wav, iso_sr))
    
    return label_torch, reg_wav_list, iso_wav_list

label_torch, reg_wav_list, iso_wav_list = zip_label_wav()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([38, 120])
Loading file: ABoyNamedSue.wav
Loading file: AintNoGrave.wav


KeyboardInterrupt: 

# Note for Andy

for label_torch, the size is (38, 120)
if you need [1, 120] tensor each time, just do
label_torch[i].unsqueeze(0)